In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [181]:
logits = tf.random.uniform(minval=0, maxval=1, seed=42, shape=(5, 6))
logits

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.70125747, 0.59246445, 0.6469835 , 0.78929853, 0.4893242 ,
        0.3564489 ],
       [0.87974083, 0.8299811 , 0.4734974 , 0.06678545, 0.37019384,
        0.8914565 ],
       [0.9230031 , 0.97236   , 0.9784479 , 0.99113977, 0.6481546 ,
        0.04302442],
       [0.26199007, 0.6073632 , 0.41687417, 0.33327353, 0.5742605 ,
        0.2488842 ],
       [0.8426975 , 0.03221571, 0.45767534, 0.19128656, 0.23360837,
        0.9743171 ]], dtype=float32)>

In [182]:
Q_marginal = tfd.MixtureSameFamily(
mixture_distribution=tfd.Categorical(logits=tf.zeros(tf.shape(logits)[0])),
    components_distribution=tfd.RelaxedBernoulli(logits=tf.transpose(logits),
                                                 temperature=1e-5)
)
Q_marginal

<tfp.distributions.MixtureSameFamily 'MixtureSameFamily' batch_shape=[6] event_shape=[] dtype=float32>

In [183]:
x = Q_marginal.sample(512)
x = tf.clip_by_value(x, 1e-7, 1.-1e-7)
tf.reduce_mean(-1. * tf.reduce_sum(Q_marginal.log_prob(x), axis=1))


<tf.Tensor: shape=(), dtype=float32, numpy=-18.08369>

In [186]:
tf.reduce_mean(-1. * Q_marginal.log_prob(x))

<tf.Tensor: shape=(), dtype=float32, numpy=-3.0139532>

In [154]:
tfd.RelaxedBernoulli(logits=logits[0], temperature=1e-5).sample(5)

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[1.69425517e-01, 2.86047843e-08, 9.99991059e-01, 1.26213822e-11,
        9.84694362e-01, 9.80800986e-01],
       [1.12091314e-07, 9.99999821e-01, 1.00000000e+00, 9.99950886e-01,
        9.99999762e-01, 9.99994159e-01],
       [9.99905467e-01, 1.00000000e+00, 9.99999762e-01, 1.87024474e-03,
        7.75832814e-06, 1.00000000e+00],
       [1.64315952e-06, 1.31407887e-06, 9.98902619e-01, 2.60528028e-02,
        1.00000000e+00, 9.99572515e-01],
       [8.47551376e-02, 7.52730131e-01, 2.94035551e-09, 9.84307036e-08,
        9.19312119e-01, 9.39749181e-01]], dtype=float32)>

In [46]:
tf.reshape(tf.repeat(tf.one_hot(tf.range(5), 5), 4, axis=0), shape=(5, 4, 5))

<tf.Tensor: shape=(5, 4, 5), dtype=float32, numpy=
array([[[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.]],

       [[0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]],

       [[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]], dtype=float32)>

In [40]:
logits = tf.random.uniform(shape=(4, 6))

In [39]:
tf.stack([logits] * 5)

<tf.Tensor: shape=(5, 4, 6), dtype=float32, numpy=
array([[[0.43997335, 0.7566391 , 0.72414565, 0.8721417 , 0.77916086,
         0.8345915 ],
        [0.8838308 , 0.9762491 , 0.9743644 , 0.60298705, 0.9274093 ,
         0.16422224],
        [0.16155815, 0.40403342, 0.6110437 , 0.7467438 , 0.37594604,
         0.8383919 ],
        [0.1807034 , 0.78149307, 0.10201752, 0.9994273 , 0.84367716,
         0.97526073]],

       [[0.43997335, 0.7566391 , 0.72414565, 0.8721417 , 0.77916086,
         0.8345915 ],
        [0.8838308 , 0.9762491 , 0.9743644 , 0.60298705, 0.9274093 ,
         0.16422224],
        [0.16155815, 0.40403342, 0.6110437 , 0.7467438 , 0.37594604,
         0.8383919 ],
        [0.1807034 , 0.78149307, 0.10201752, 0.9994273 , 0.84367716,
         0.97526073]],

       [[0.43997335, 0.7566391 , 0.72414565, 0.8721417 , 0.77916086,
         0.8345915 ],
        [0.8838308 , 0.9762491 , 0.9743644 , 0.60298705, 0.9274093 ,
         0.16422224],
        [0.16155815, 0.40403342, 0.

In [42]:
tf.stack([tf.one_hot(tf.range(5), 5)] * 5)

<tf.Tensor: shape=(5, 5, 5), dtype=float32, numpy=
array([[[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]], dtype=float32)>

In [64]:
tf.reshape(tf.repeat(tf.expand_dims(tf.one_hot(2, 5), axis=0), 3, axis=0), shape=(3, 5))

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

In [60]:
tf.stack([tf.one_hot(2, 5)] * 3)

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

In [4]:
latent_state_size = 3
batch_size = 6
action_size = 2

action = tf.random.uniform(
    shape=(batch_size, ), minval=0, maxval=action_size, dtype=tf.int32, seed=42)
action = tf.one_hot(action, action_size)

latent_states_logits = []
for _ in range(action_size):
    latent_states_logits.append(
        tf.random.uniform(shape=(batch_size, latent_state_size), seed=42))
# latent_states_stack = tf.stack(latent_states_logits, axis=1)

latent_states_stack = tf.random.uniform(shape=(batch_size, action_size, latent_state_size))
action_expand_dims = tf.expand_dims(action, -1)
action_tile = tf.tile(action_expand_dims, (1, 1, latent_state_size))
mult = action_tile * latent_states_stack
latent_states = tf.reduce_sum(mult, axis=1)

print("action", action)
print("latent_states_stack", latent_states_stack)
print("action_tile", action_tile)
print("mult", mult)
print(latent_states)

action tf.Tensor(
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]], shape=(6, 2), dtype=float32)
latent_states_stack tf.Tensor(
[[[0.29822934 0.83325636 0.142084  ]
  [0.81573653 0.8939357  0.21609187]]

 [[0.66496444 0.90702116 0.05573988]
  [0.4616239  0.8716624  0.21884048]]

 [[0.3344425  0.51248133 0.19966173]
  [0.7560613  0.01114261 0.44287622]]

 [[0.5316609  0.12373877 0.35417366]
  [0.6469995  0.06985843 0.6044067 ]]

 [[0.8172021  0.31625688 0.7307626 ]
  [0.3524387  0.87245095 0.68590915]]

 [[0.19558418 0.15751529 0.39862943]
  [0.20925236 0.28273594 0.54019547]]], shape=(6, 2, 3), dtype=float32)
action_tile tf.Tensor(
[[[1. 1. 1.]
  [0. 0. 0.]]

 [[1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]]], shape=(6, 2, 3), dtype=float32)
mult tf.Tensor(
[[[0.29822934 0.83325636 0.142084  ]
  [0.         0.         0.        ]]

 [[0.66496444 0.90702116 0.05573988]
  [0.         0.        

In [27]:
print(tf.tile(action_expand_dims, (1, 1, latent_state_size)))
print(tf.reshape(tf.repeat(action_expand_dims, latent_state_size), shape=(batch_size, action_size, latent_state_size)))
print(tf.repeat(action_expand_dims, latent_state_size, axis=-1))

tf.Tensor(
[[[0. 0. 0.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]]

 [[1. 1. 1.]
  [0. 0. 0.]]], shape=(6, 2, 3), dtype=float32)
tf.Tensor(
[[[0. 0. 0.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]]

 [[1. 1. 1.]
  [0. 0. 0.]]], shape=(6, 2, 3), dtype=float32)


<tf.Tensor: shape=(6, 2, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [0., 0., 0.]]], dtype=float32)>

In [9]:
layers = []
# layers.append(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, -1)))
layers.append(tf.keras.layers.RepeatVector(latent_state_size))
layers.append(tf.keras.layers.Permute(dims=(2, 1)))
sequential = tf.keras.Sequential(layers)
sequential(action)

<tf.Tensor: shape=(6, 2, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [0., 0., 0.]]], dtype=float32)>